In [1]:
# create a spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").\
                                     appName("spark_on_docker").\
                                     getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/20 01:43:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Fault Tolerance and Checkpointing

In a failure scenario, you simply need to restart your application, making sure to point to the same checkpoint location, and it will automatically recover its state and start processing data where it left off. 

In [2]:
# in Python
spark.conf.set("spark.sql.shuffle.partitions", 5)

static = spark.read.json("work/TheDefinitiveGuide/Spark-The-Definitive-Guide/data/activity-data")

streaming = spark\
    .readStream\
    .schema(static.schema)\
    .option("maxFilesPerTrigger", 10)\
    .json("work/TheDefinitiveGuide/Spark-The-Definitive-Guide/data/activity-data")\
    .groupBy("gt")\
    .count()

query = streaming\
    .writeStream\
    .outputMode("complete")\
    .option("checkpointLocation", "work/")\
    .queryName("test_python_stream")\
    .format("memory")\
    .start()

22/01/20 01:43:56 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


root
 |-- gt: string (nullable = true)
 |-- count: long (nullable = false)



22/01/20 01:44:03 WARN FileStreamSource: Listed 80 file(s) in 5515 ms


In [3]:
streaming.printSchema()  

root
 |-- gt: string (nullable = true)
 |-- count: long (nullable = false)



Query Status

The query status is the most basic monitoring API, so it’s a good starting point. It aims to answer the question, “What processing is my stream performing right now?” This information is reported in the status field of the query object returned by startStream. For example, you might have a simple counts stream that provides counts of IOT devices defined by the following query (here we’re just using the same query from the previous chapter without the initialization code):


In [ ]:
query.status

Recent Progress

While the query’s current status is useful to see, equally important is an ability to view the query’s progress. The progress API allows us to answer questions like “At what rate am I processing tuples?” or “How fast are tuples arriving from the source?” By running query.recentProgress, you’ll get access to more time-based information like the processing rate and batch durations. The streaming query progress also includes information about the input sources and output sinks behind your stream.

In [8]:
query.recentProgress

22/01/20 04:26:26 WARN FileStreamSource: Listed 80 file(s) in 6013 ms


[{'id': 'c27d63a7-4850-4d07-b3e6-d4b82638cd31',
  'runId': 'c47aa7dd-a984-458f-8971-1b6cbaba5f63',
  'name': 'test_python_stream',
  'timestamp': '2022-01-20T04:06:54.096Z',
  'batchId': 8,
  'numInputRows': 0,
  'inputRowsPerSecond': 0.0,
  'processedRowsPerSecond': 0.0,
  'durationMs': {'latestOffset': 6107, 'triggerExecution': 6107},
  'stateOperators': [{'operatorName': 'stateStoreSave',
    'numRowsTotal': 7,
    'numRowsUpdated': 0,
    'allUpdatesTimeMs': 932,
    'numRowsRemoved': 0,
    'allRemovalsTimeMs': 0,
    'commitTimeMs': 2433,
    'memoryUsedBytes': 4280,
    'numRowsDroppedByWatermark': 0,
    'numShufflePartitions': 5,
    'numStateStoreInstances': 5,
    'customMetrics': {'loadedMapCacheHitCount': 70,
     'loadedMapCacheMissCount': 0,
     'stateOnCurrentVersionSizeBytes': 1864}}],
  'sources': [{'description': 'FileStreamSource[file:/home/jovyan/work/TheDefinitiveGuide/Spark-The-Definitive-Guide/data/activity-data]',
    'startOffset': {'logOffset': 7},
    'endO

22/01/20 04:26:35 WARN FileStreamSource: Listed 80 file(s) in 9040 ms
22/01/20 04:26:41 WARN FileStreamSource: Listed 80 file(s) in 5700 ms
22/01/20 04:26:47 WARN FileStreamSource: Listed 80 file(s) in 5968 ms
22/01/20 04:26:54 WARN FileStreamSource: Listed 80 file(s) in 6771 ms
22/01/20 04:27:00 WARN FileStreamSource: Listed 80 file(s) in 6003 ms
22/01/20 04:27:05 WARN FileStreamSource: Listed 80 file(s) in 5323 ms
22/01/20 04:27:10 WARN FileStreamSource: Listed 80 file(s) in 5290 ms
22/01/20 04:27:17 WARN FileStreamSource: Listed 80 file(s) in 6296 ms
22/01/20 04:27:22 WARN FileStreamSource: Listed 80 file(s) in 5541 ms
22/01/20 04:27:27 WARN FileStreamSource: Listed 80 file(s) in 5149 ms
22/01/20 04:27:33 WARN FileStreamSource: Listed 80 file(s) in 5065 ms
22/01/20 04:27:38 WARN FileStreamSource: Listed 80 file(s) in 5413 ms
22/01/20 04:27:44 WARN FileStreamSource: Listed 80 file(s) in 5511 ms
22/01/20 04:27:49 WARN FileStreamSource: Listed 80 file(s) in 5229 ms
22/01/20 04:27:54 WA